# Importing langchain community library

In [1]:
# Run installation once by uncommenting the below
# !pip3 install langchain_community
# !pip3 install pandas
import re
import csv
import calendar

# Importing Ollama Model

In [2]:
from langchain_community.llms import Ollama

/Users/mallelasandeep/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Starting Using llama2 large language model

In [3]:
llm = Ollama(model="llama2")
# example usage
llm.invoke("Hello!")

"\nHello there! It's nice to meet you. How are you today?"

# Reading Income Transactions

In [4]:
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()
transactions = df
transactions

,Date,Name / Description,Expense/Income,Amount (INR)
0,01/14/2022,Audible Uk AdblCo/Pymt Gbr,Expense,11.00
1,01/15/2022,Classpass* Monthly,Expense,30.08
2,01/16/2022,Amazon Lux,Expense,24.11
3,01/31/2022,Salary,Income,796911.67
4,02/28/2022,Salary,Income,1.00
5,02/28/2022,Salary,Income,716320.00
6,04/30/2022,Salary,Income,800000.00
7,08/17/2022,Salary,Income,780542.56
8,10/21/2022,Tesco Breda,Expense,22.80
9,10/22/2022,Apple Services,Expense,41.25


# Batching transactions and running through LLM

In [19]:
missing_credits_df = pd.DataFrame(columns=['DepositAmount', 'Income'])
def identify_missing_transactions(all_transactions, llm, batch_size):
    global missing_credits_df
    num_batches = len(all_transactions) // batch_size
    if len(all_transactions) % batch_size != 0:
        num_batches += 1

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(all_transactions))
        batch = all_transactions[start_idx:end_idx]
        print(f"{batch.size}")
        print(batch)
        transactions_str = '\n'.join(batch)
        message = " Analyse transactions with 'Name / Description' marked value marked as 'Salary' and give months where salary is not credited. Every month from Feb 2022 to Till date we are supposed have a transaction recorded for Salary.  \n" + transactions_str
        response = llm.invoke(message)
        response = response.split('\n')
        print(response)
        return response


# Building missing transaction identifier

In [ ]:
def identify_missing_transactions(all_transactions, llm):
    transactions_str = '\n'.join(all_transactions)
    message = "List out missing months where income is not registered \n" + transactions_str
    response = llm.invoke(message)
    response = response.split('\n')
    return response

## Running identifier

In [20]:
# print(transactions)

identify_missing_transactions(transactions, llm, 5)
# print(response)

20
         Date          Name / Description Expense/Income  Amount (INR)
0  01/14/2022  Audible Uk AdblCo/Pymt Gbr        Expense         11.00
1  01/15/2022          Classpass* Monthly        Expense         30.08
2  01/16/2022                  Amazon Lux        Expense         24.11
3  01/31/2022                      Salary         Income     796911.67
4  02/28/2022                      Salary         Income          1.00


 # Writing Missing Income months to CSV

In [ ]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)